In [1]:
!python3 -m pip install pandas
!python3 -m pip install sklearn
!pip install ray[tune]

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import sklearn

# Your task now is to use scikit-learn to train a RandomForestClassifier https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html  for predicting the forest cover type. The dataset can be downloaded directly via the sklearn API: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html#sklearn.datasets.fetch_covtype and there are several notebooks on Kaggle where you can learn about the data and background to the dataset: https://www.kaggle.com/c/forest-cover-type-prediction.

# Do the following:
# 1.	Set up the problem using sklearn. Explore the parameters used in the Random Forest implementation. Hint: use the get_param() method. Train the model using the default parameters.

# 2.	Implement a distributed hyperparameter tuning pipeline for tuning the parameters ‘max_depth’, ‘n_estimators’ and ‘ccp_alpha’ using the RayTune framework and an exhaustive Grid Search strategy. Configure, deploy and run it on up to 3 VMs of “small” flavor.

In [3]:
trainDF = pd.read_csv('train.csv')
#testDF = pd.read_csv('test.csv') - this file doesn't actually contain the target variable of soil type.

trainDF.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [4]:
trainFeatures = trainDF.iloc[:, :-1] # all rows, all but last column
trainClasses = trainDF.iloc[:, -1] # all rows, only last column

In [5]:
RFclassifier = RandomForestClassifier()
params = RFclassifier.get_params()

In [6]:
#fitted_model = RFclassifier.fit(trainFeatures, trainClasses)

sklearn.model_selection.cross_validate(RFclassifier, trainFeatures, trainClasses)

{'fit_time': array([2.19727921, 2.14854646, 2.11283231, 2.1297574 , 2.02893901]),
 'score_time': array([0.06794953, 0.07835221, 0.09362078, 0.08149385, 0.08172727]),
 'test_score': array([0.4457672 , 0.6087963 , 0.69246032, 0.73875661, 0.76785714])}

In [7]:
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [8]:
ray.init(object_store_memory=ray.ray_constants.OBJECT_STORE_MINIMUM_MEMORY_BYTES)

2022-04-29 00:20:16,085	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8267


RayContext(dashboard_url='127.0.0.1:8267', python_version='3.8.10', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '192.168.2.213', 'raylet_ip_address': '192.168.2.213', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-04-29_00-20-12_314705_54437/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-04-29_00-20-12_314705_54437/sockets/raylet', 'webui_url': '127.0.0.1:8267', 'session_dir': '/tmp/ray/session_2022-04-29_00-20-12_314705_54437', 'metrics_export_port': 55226, 'gcs_address': '192.168.2.213:60861', 'address': '192.168.2.213:60861', 'node_id': '22f8345a55701a84d0c4b2e064abf00aaa821ca4804fe8fbad1ab1fe'})

In [9]:
search_space = {
    "max_depth": tune.grid_search([5*i for i in range(1,10)]),
    "n_estimators": tune.grid_search([10,100,200,300]),
    "ccp_alpha": tune.grid_search([0.0,0.1,0.2]),
}

In [10]:
import statistics

def train(config):
    RFclassifier = RandomForestClassifier(**config)
    params = RFclassifier.get_params()
    xval = sklearn.model_selection.cross_validate(RFclassifier, trainFeatures, trainClasses)
    tune.report(mean_accuracy=statistics.mean(xval["test_score"]))
    

In [ ]:
analysis = tune.run(train, config=search_space)

2022-04-29 00:20:19,752	WARNING function_runner.py:598 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-04-29 00:20:21,803	INFO trial_runner.py:803 -- starting train_27206_00000


Trial name,status,loc,ccp_alpha,max_depth,n_estimators
train_27206_00000,RUNNING,192.168.2.213:54649,0,5,10
train_27206_00001,PENDING,,0.1,5,10
train_27206_00002,PENDING,,0.2,5,10
train_27206_00003,PENDING,,0,10,10
train_27206_00004,PENDING,,0.1,10,10
train_27206_00005,PENDING,,0.2,10,10
train_27206_00006,PENDING,,0,15,10
train_27206_00007,PENDING,,0.1,15,10
train_27206_00008,PENDING,,0.2,15,10
train_27206_00009,PENDING,,0,20,10


Result for train_27206_00000:
  date: 2022-04-29_00-20-28
  done: false
  experiment_id: 573b0070288f446fba295d355443999b
  hostname: ben-ray-small
  iterations_since_restore: 1
  mean_accuracy: 0.5753306878306879
  node_ip: 192.168.2.213
  pid: 54649
  time_since_restore: 0.7262163162231445
  time_this_iter_s: 0.7262163162231445
  time_total_s: 0.7262163162231445
  timestamp: 1651191628
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '27206_00000'
  warmup_time: 0.013929128646850586
  


Trial name,status,loc,ccp_alpha,max_depth,n_estimators,acc,iter,total time (s)
train_27206_00000,RUNNING,192.168.2.213:54649,0,5,10,0.575331,1,0.726216
train_27206_00001,PENDING,,0.1,5,10,,,
train_27206_00002,PENDING,,0.2,5,10,,,
train_27206_00003,PENDING,,0,10,10,,,
train_27206_00004,PENDING,,0.1,10,10,,,
train_27206_00005,PENDING,,0.2,10,10,,,
train_27206_00006,PENDING,,0,15,10,,,
train_27206_00007,PENDING,,0.1,15,10,,,
train_27206_00008,PENDING,,0.2,15,10,,,
train_27206_00009,PENDING,,0,20,10,,,


Result for train_27206_00000:
  date: 2022-04-29_00-20-28
  done: true
  experiment_id: 573b0070288f446fba295d355443999b
  experiment_tag: 0_ccp_alpha=0.0,max_depth=5,n_estimators=10
  hostname: ben-ray-small
  iterations_since_restore: 1
  mean_accuracy: 0.5753306878306879
  node_ip: 192.168.2.213
  pid: 54649
  time_since_restore: 0.7262163162231445
  time_this_iter_s: 0.7262163162231445
  time_total_s: 0.7262163162231445
  timestamp: 1651191628
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '27206_00000'
  warmup_time: 0.013929128646850586
  


...

2022-04-29 00:24:00,754	WARNING util.py:171 -- The `on_step_begin` operation took 1.029 s, which may be a performance bottleneck.


Trial name,status,loc,ccp_alpha,max_depth,n_estimators,acc,iter,total time (s)
train_27206_00006,RUNNING,192.168.2.213:55095,0,15,10,,,
train_27206_00007,PENDING,,0.1,15,10,,,
train_27206_00008,PENDING,,0.2,15,10,,,
train_27206_00009,PENDING,,0,20,10,,,
train_27206_00010,PENDING,,0.1,20,10,,,
train_27206_00011,PENDING,,0.2,20,10,,,
train_27206_00012,PENDING,,0,25,10,,,
train_27206_00013,PENDING,,0.1,25,10,,,
train_27206_00014,PENDING,,0.2,25,10,,,
train_27206_00015,PENDING,,0,30,10,,,


Trial name,status,loc,ccp_alpha,max_depth,n_estimators,acc,iter,total time (s)
train_27206_00006,RUNNING,192.168.2.213:55095,0,15,10,,,
train_27206_00007,PENDING,,0.1,15,10,,,
train_27206_00008,PENDING,,0.2,15,10,,,
train_27206_00009,PENDING,,0,20,10,,,
train_27206_00010,PENDING,,0.1,20,10,,,
train_27206_00011,PENDING,,0.2,20,10,,,
train_27206_00012,PENDING,,0,25,10,,,
train_27206_00013,PENDING,,0.1,25,10,,,
train_27206_00014,PENDING,,0.2,25,10,,,
train_27206_00015,PENDING,,0,30,10,,,


2022-04-29 00:24:15,650	WARNING util.py:171 -- The `on_step_begin` operation took 0.754 s, which may be a performance bottleneck.


Trial name,status,loc,ccp_alpha,max_depth,n_estimators,acc,iter,total time (s)
train_27206_00006,RUNNING,192.168.2.213:55095,0,15,10,,,
train_27206_00007,PENDING,,0.1,15,10,,,
train_27206_00008,PENDING,,0.2,15,10,,,
train_27206_00009,PENDING,,0,20,10,,,
train_27206_00010,PENDING,,0.1,20,10,,,
train_27206_00011,PENDING,,0.2,20,10,,,
train_27206_00012,PENDING,,0,25,10,,,
train_27206_00013,PENDING,,0.1,25,10,,,
train_27206_00014,PENDING,,0.2,25,10,,,
train_27206_00015,PENDING,,0,30,10,,,
